In [ ]:
import pandas as pd
import numpy as np
import matplotlib.colors as mcolors
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # suppress some future warning

from os import path
from scipy.stats import poisson # TODO sostituisci con smf o sm?
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

# custom libs
from libs.utils import fix_continent_matches, show_cdf, show_cdf_splitted, format_dataframe, extract_continent_from_FIFA, check_records_validity, remove_outliers, get_match_result
# from libs.utils import convert_onehot, convert_onehot_simplified TODO al momento non sono utilizzate, eventualmente elimina

plt.style.use('ggplot') # stile di default del grafico
# plt.rcParams.update({'figure.figsize':(45,10), 'figure.dpi':100})

# costanti
PATH_ORIGINAL_DST = 'dataset/original'
PATH_DST = 'dataset'

## Parte 1: Introduzione dataset, verifica e preparazione

Si prende in analisi un dataset contenente 4483 risultati di partite di calcio internazionale maschile.<br>
L'obbiettivo è realizzare un predittore di risultati di partite di calcio internazionale [... aggiungi descrizione dal dataset stesso...]. Inizialmente viene preso in esame il dataset, vengono estratte alcune informazioni utilizzando la statistica descrittiva.<br>
Per prima cosa verifico l'eventuale presenza di valori nulli del dataset

In [ ]:
df = pd.read_csv(path.join(PATH_ORIGINAL_DST ,'results.csv') )

print("Null values:", df.isnull().values.any(), "| shape:", df.shape)
print("***"*15)
print(df.info())

### Pre-processing
Dopo essermi accertato che il dataset non contiene elementi nulli procedo con il pre-processing:
- Estraggo la data dalla colonna `date` in rispettivamente in `day` `day_of_week` `month` `year`:
- Calcolo `outcome` dall'incontro utilizzando la funzione `return_outcome` che ritorna `Home` per vittoria della squadra di casa, `Draw` per il pareggio e `Away` per vittoria della squadra ospite
- Converto il valore `home_score` e `away_score` in `float` (servirà successivamente)
- Alcuni nomi di nazioni sono cambiate nel tempo, in quanto il dataset presenta incontri che comprendono ad ese
- Formatto i nomi delle nazioni [scrivi meglio]

In [ ]:
df = format_dataframe(df)

Solitamente, nel calcio, la prima squadra è considerata sempre quella di casa. Controllo dunque che il dataset sia coerente con questa convenzione prima di procedere al calcolo delle statistiche per ogni team

In [ ]:
if len(df.query("away_team == country")) > 0:
    print("Trovati %d record incorenti con la convenzione" % len(df.query("away_team == country")))

Visto l'esiguo numero di elementi li visualizzo

In [ ]:
df.query("away_team == country")

Sono state trovate dunque 25 incoerenze riguardo alla convenzione spiegata sopre. È semplice spiegarle:
- I record che presentano neutral = False: è spiegato dal fatto che la Silesia è oggi parte della Polonia e della Republica Ceca, mentre la Catalonia ad oggi è annessa alla Spagna. Questo sta a dimostrare che sono presenti  dati 'spuri' dal dataset di cui è già stata prevista l'eliminazione in seguito,<br>
- I record che presentano neutral = True sono incoerenti<br>

Sistemo:

In [ ]:
for i in df.query("away_team == country and neutral == True").index:
    df.at[i, "neutral"] = False

Controllo che la presenza di ulteriori incoerenze anche nel caso in cui `home_team` sia uguale a `contry` e `neutral` sia uguale a `True`

In [ ]:
df.query("home_team == country and neutral == True")

In [ ]:
for i in df.query("home_team == country and neutral == True").index:
    df.at[i, "neutral"] = False

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

def swap_outcome(val):
    val.replace({"Home": "Away", "Away":"Home"})
    return val

neutral_games = df.query("neutral == True")
not_neutral_games = df.query("neutral == False")

invalid_ = not_neutral_games.query("away_team == country")

invalid_copy = invalid_.copy()

invalid_.loc[invalid_.index.values, 'home_team'] = invalid_copy.loc[invalid_.index.values, 'away_team']
invalid_.loc[invalid_.index.values, 'home_score'] = invalid_copy.loc[invalid_.index.values, 'away_score'] 
invalid_.loc[invalid_.index.values, 'away_team'] = invalid_copy.loc[invalid_.index.values, 'home_team'] 
invalid_.loc[invalid_.index.values, 'away_score'] = invalid_copy.loc[invalid_.index.values, 'home_score'] 
invalid_['outcome'] = invalid_['outcome'].replace({"Home": "Away", "Away":"Home"})

for i in invalid_.index.values:
    df.loc[i, 'home_team'] = invalid_.loc[i, 'home_team']
    df.loc[i, 'home_score'] = invalid_.loc[i, 'home_score']
    df.loc[i, 'away_team'] = invalid_.loc[i, 'away_team']
    df.loc[i, 'away_score'] = invalid_.loc[i, 'away_score']
    df.loc[i, 'outcome'] = invalid_.loc[i, 'outcome']

In [ ]:
df.to_csv(path.join(PATH_DST, 'dataset_v1.csv'))

### Check validità match

Controllo la presenza di eventuali record ambigui, data anche la presenza di partite vecchie, ad esempio di nazioni non più presenti... spiega meglio ... non li considero da subito.

Per validare le squadre utilizzerò il dataset FIFA ...descrivi....

In [ ]:
len(df)

In [ ]:
df_fifa = pd.read_csv(path.join(PATH_ORIGINAL_DST, 'fifa_ranking-2021-05-27.csv'))

df = check_records_validity(df, df_fifa)

df_valid = df.query("is_valid == True").drop(columns=['is_valid']).reset_index(drop=True)
df_valid.to_csv(path.join(PATH_DST, "dataset_v1_valid.csv"))
df_not_valid = df.query("is_valid == False").drop(columns=['is_valid']).reset_index(drop=True)
df_not_valid.to_csv(path.join(PATH_DST, "dataset_v1_not_valid.csv"))

if len(df_valid) + len(df_not_valid) == len(df):
    print("valid: %s not valid: %s, totale di record esaminati %s" % ( len(df_valid),len(df_not_valid), len(df) ) )
else:
    print("Something wrong happened!")

### Check validità nazioni

Dovrebbe servire per dopo nello specifico ma dato che questa è la parte dove pulisco il dataset lo faccio qui...

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'
df_valid_country = pd.read_csv(path.join(PATH_DST, 'dataset_v1_valid.csv')).drop(columns=['Unnamed: 0'])

df_fifa = pd.read_csv(path.join(PATH_ORIGINAL_DST, 'fifa_ranking-2021-05-27.csv')).drop(columns=['id','rank', 'country_abrv', 'total_points', 'previous_points', 'rank_change', 'rank_date']).sort_values(by='country_full').drop_duplicates().reset_index(drop=True)

df_unvalid_country = pd.DataFrame()

# estraggo tutti i team presenti in dataset_reduced_fixed
df_ht = df_valid_country['home_team'].drop_duplicates().sort_values().reset_index().drop(labels=['index'], axis=1).rename(columns={'home_team': 'team'})
df_at = df_valid_country['away_team'].drop_duplicates().sort_values().reset_index().drop(labels=['index'], axis=1).rename(columns={'away_team': 'team'})

# controllo che il numero di squadre sia lo stesso
if df_ht.__len__() != df_at.__len__():
    print("Not same length! Some mismatch")



df_valid_country = pd.concat([df_ht,df_at]).drop_duplicates().sort_values(by='team').reset_index().drop(labels=['index'], axis=1)

df_valid_country['confederation'] = ""

for i, row in df_valid_country.iterrows():
    if(len(df_fifa.query("country_full == @row.team")) > 0):
        df_valid_country.at[i, 'confederation'] = df_fifa.query("country_full == @row.team").confederation.values[0]

df_valid_country.replace("", np.nan, inplace=True)

df_unvalid_country = df_valid_country[df_valid_country['confederation'].isna()]
df_unvalid_country.reset_index(drop=True, inplace=True)
df_unvalid_country.drop(labels=['confederation'], axis=1, inplace=True)

# utilizzo le squadre non presenti in FIFA ranking
df_valid_country.dropna(subset = ['confederation'], inplace=True)
df_valid_country.reset_index(drop=True, inplace=True)

print("... Fixed. Number of teams: %d" % len(df_valid_country))

# salvo i dataframe perché potrebbero servire successivamente
df_valid_country.to_csv(path.join(PATH_DST, 'dataset_v1_valid_country.csv'))
df_unvalid_country.to_csv(path.join(PATH_DST, 'dataset_v1_not_valid_country.csv'))

Procedo con eliminare tutte quei match che presentano squadre in 'df_unvalid_countries_csv' e realizzo la seconda versione del dataset

In [ ]:
# per evitare problemi riguardo le variabili reimporto i csv
df = pd.read_csv(path.join(PATH_DST, 'dataset_v1_valid.csv')).drop(columns=['Unnamed: 0'])

for index, row in df_unvalid_country.iterrows():
    if(len(df.query("home_team == @row.team or away_team == @row.team")) > 0):
        df.drop(df.query("home_team == @row.team or away_team == @row.team").index.values, inplace=True)
    

print("Now dataset is formed by: %d records"% len(df))

Controllo che il numero di squadre che ha giocato fuori casa è lo stesso di chi ha giocato in casa ed eventualmente visualizzo

In [ ]:
if len(df['home_team'].drop_duplicates().sort_values().values) > len(df['away_team'].drop_duplicates().sort_values().values):
    print("Mismatch home_team > away_team: %s" % list(set(df['home_team'].drop_duplicates().sort_values().values).difference(df['away_team'].drop_duplicates().sort_values().values)))
elif len(df['home_team'].drop_duplicates().sort_values().values) < len(df['away_team'].drop_duplicates().sort_values().values):
    print("Mismatch home_team < away_team: %s" % list(set(df['away_team'].drop_duplicates().sort_values().values).difference(df['home_team'].drop_duplicates().sort_values().values)))
else:
    print("Home teams and away teams are the same")

In [ ]:
# print("Deleting %s records from dataset..."% len(df[df.home_team == "St. Kitts and Nevis"].index) )
# df = df.drop(df[df.home_team == "St. Kitts and Nevis"].index).reset_index(drop=True)
# print("Now dataset is formed by: %d records"% len(df))
df = df.reset_index(drop=True)
df.to_csv(path.join(PATH_DST, "dataset_v2.csv"))


## Parte 2: Statistica descrittiva - Distribuzione dei dati
Una volta effettuato il preprocessing del dataset vengono applicati alcuni studi. In ordine verrà definito, graficamente:
- Competizioni presenti nel dataset
- Numero di partite disputate per anno
- Numero di partite disputate per anno in base al continente (in modo tale da capire anche una eventuale inizio della diffusione del calcio nel mondo)
- Numero di goal per anno, divise in base al continente

Dopo aver estrapolato queste informazioni, si procede con una selezione dei migliori team durante la storia divisi per attacco/difesa goal fatti/subiti.<br>
Da questi insiemi verranno estratti una gruppo di squadre per mostrare l'andamento della squadra negli anni.<br>
Inoltre verrà controllato se questo insieme di squadre hanno più propensione di vincere a casa o in trasferta.<br>

<b>Per questo 'capitolo' pendo come riferimento il dataset v2'</b>

In [ ]:
df = pd.read_csv(path.join(PATH_DST, 'dataset_v2.csv')).drop(columns=['Unnamed: 0'])

df.head(5)

### Competizioni presenti nel dataset

In [ ]:
tournaments = df['tournament'].value_counts()

print("Number of tournaments: %d" % ( len(tournaments) ))

# Visto l'elevato numero di competizioni presenti, considero solo le prime 15 in base al numero di partite.

plt.title('Frequency of competition in dataset')
plt.ylabel('Number of matches')
plt.xlabel('Competitions')
plt.bar(tournaments.index[:15], tournaments.values[:15], color=[i for i in mcolors.TABLEAU_COLORS], edgecolor='black')

plt.show()

### Numero di partite disputate per anno

In [ ]:
# matches per year
mpy = df['year'].value_counts().sort_index()

plt.figure(figsize=(15,4))
plt.title(label='Number of games per year')
plt.ylabel(ylabel='number of games')
plt.xlabel(xlabel='years')

l1 = [(1914,0), (1914,1200)]
l2 = [(1918,0), (1918,1200)]
l3 = [(1939,0), (1939,1200)]
l4 = [(1945,0), (1945,1200)]
l5 = [(2019,0), (2019, 1200)]

lc = LineCollection([l1, l2, l3, l4, l5], color=["orangered","orangered", "crimson","crimson", "darkcyan"], lw=2, linestyles='-.')

plt.annotate('1st World War', xy=(1914, 200), xycoords='data', ha='center', xytext=(-40, -20), textcoords='offset points', color='orangered')
plt.annotate('2nd World War', xy=(1939, 200), xycoords='data', ha='center', xytext=(-40, -30), textcoords='offset points', color='crimson')
plt.annotate('COVID-19', xy=(2019, 200), xycoords='data', ha='center', xytext=(-40, -20), textcoords='offset points', color='darkcyan')

plt.gca().add_collection(lc)

plt.plot(mpy, color='royalblue')
plt.grid(True)
plt.show()

Si nota che il numero di match si è abbassato durante le Guerre Mondiali ed è drasticamente calato durante la pandemia di Covid-19 ad inizio 2020.

### Numero di partite giocate per anno in ogni continente
Dal dataset FIFA carico le confederazioni delle nazionali. Ricavo dunque i continenti in cui è stato disputato il match

In [ ]:
df_fifa = pd.read_csv(path.join(PATH_ORIGINAL_DST,'fifa_ranking-2021-05-27.csv'))

df = extract_continent_from_FIFA(df, df_fifa)

In [ ]:
# dataframe continent reduced
df_cr = df.groupby(['continent','year']).size().to_csv('output.csv') # risolvo il bug del groupby che non scrive in tutte le colonne i continenti
df_cr = pd.read_csv('output.csv')

df_cr.sort_values(by='year', axis=0, inplace=True)
df_cr = df_cr.reset_index().drop(columns=['index']).rename(columns={'0': 'matches'})

if df.__len__() == df_cr['matches'].sum():
    print("Groupby eseguito correttamente")

years = df_cr['year'].drop_duplicates().to_numpy()

africa = df_cr[df_cr['continent'] == 'Africa' ].reset_index().drop(columns=['continent', 'index'])
america = df_cr[df_cr['continent'] == 'America'].reset_index().drop(columns=['continent', 'index'])
asia = df_cr[df_cr['continent'] == 'Asia'].reset_index().drop(columns=['continent', 'index'])
europe = df_cr[df_cr['continent'] == 'Europe'].reset_index().drop(columns=['continent', 'index'])
oceania = df_cr[df_cr['continent'] == 'Oceania'].reset_index().drop(columns=['continent', 'index'])

print("First appearance in national match:\nAfrica: %d\nAmerica: %d\nAsia: %d\nEurope: %d\nOceania: %d" % (africa['year'].min(), america['year'].min(), asia['year'].min(), europe['year'].min(), oceania['year'].min() ))
africa  =  fix_continent_matches(years, africa)
america =  fix_continent_matches(years, america)
asia     = fix_continent_matches(years, asia)
europe  =  fix_continent_matches(years, europe)
oceania =  fix_continent_matches(years, oceania)

if len(africa) == len(europe) == len(america) == len(oceania) == len(asia):
    print("All df have the same number of rows")

In [ ]:
plt.title('Number of games played by each continent')
plt.ylabel('Number of matches')
plt.xlabel('Years')
plt.bar(years, europe.matches.values, color='steelblue')
plt.bar(years, america.matches.values, bottom=europe.matches.values , color='crimson')
plt.bar(years, asia.matches.values, bottom = europe.matches.values+america.matches.values, color='darkorange')
plt.bar(years, oceania.matches.values, bottom = europe.matches.values+america.matches.values+asia.matches.values , color='salmon')
plt.bar(years, africa.matches.values, bottom=europe.matches.values + america.matches.values + asia.matches.values + oceania.matches.values, color='gold')
plt.legend(['Europe', 'America', 'Asia', 'Oceania', 'Africa'])

plt.show()

Dal grafico si evince che Europa ed America sono i continenti dove si sono disputato più partite. Il numero di partite disputate in oceania è irrilevante rispetto al numero degli altri continenti nonostante l'Africa abbia disputato la prima partita ufficiale per ultima. <br>
Salvo comunque il dataframe per eventuali utilizzi

In [ ]:
df.to_csv(path.join(PATH_DST, "dataset_v2_continent.csv"))

### Estrazione statistiche squadre presenti nel dataset

In [ ]:
df = pd.read_csv(path.join(PATH_DST, 'dataset_v2_continent.csv')).drop(columns=['Unnamed: 0'])
# print(df.columns)

valid_country = pd.read_csv(path.join(PATH_DST, 'dataset_v1_valid_country.csv')).drop(columns=['Unnamed: 0']).team.values

df_country = df.country.drop_duplicates().values

# print(df_country)

not_valid_country = []

for i in df_country:
    if i not in valid_country:
        not_valid_country.append(i)
# print(not_valid_country)

# trovati altri record non validi. Li elimino

print("Lunghezza del dataframe prima del controllo: %s" % (len(df)) )

for i, row in df.iterrows():
    if(row.country in not_valid_country):
        df.drop(i, inplace=True)

print("Lunghezza del dataframe prima del controllo: %s" % (len(df)) )

df = df.reset_index(drop=True).to_csv(path.join(PATH_DST, 'dataset_v3.csv'))

In [ ]:
# creo un dataframe con le statistiche di ogni squadra per vedere se ci sono delle correlazioni sul giocare in casa o no nello specifico
# e nei goal subiti/segnati
df = pd.read_csv(path.join(PATH_DST, 'dataset_v3.csv')).drop(columns=['Unnamed: 0'])

teams = pd.read_csv(path.join(PATH_DST, 'dataset_v1_valid_country.csv')).drop(columns=['Unnamed: 0'])

teams['games'] = 0
teams['home_games'] = 0
teams['away_games'] = 0
teams['neutral_games'] = 0

teams['wins'] = 0
teams['home_wins'] = 0
teams['away_wins'] = 0
teams['neutral_wins'] = 0
    
teams['defeats'] = 0
teams['home_defeats'] = 0
teams['away_defeats'] = 0
teams['neutral_defeats'] = 0

teams['draws'] = 0
teams['home_draws'] = 0
teams['away_draws'] = 0
teams['neutral_draws'] = 0

teams['goals_scored'] = 0
teams['home_goals_scored'] = 0
teams['away_goals_scored'] = 0
teams['neutral_goals_scored'] = 0

teams['goals_conceded'] = 0
teams['home_goals_conceded'] = 0
teams['away_goals_conceded'] = 0
teams['neutral_goals_conceded'] = 0

counter = 0
for team in teams.team.values:
    print('*** Processing %d/%d ***' % (counter, len(teams.team.values)), end='\r')

    team_df = df.query("home_team == @team or away_team == @team")

    home_games_df = team_df.query("neutral == False and country == @team ")
    away_games_df = team_df.query("neutral == False and country != @team ")
    neutral_games_df = team_df.query("neutral == True ")

    teams.at[teams.team == team, 'games'] = len(team_df)
    teams.at[teams.team == team, 'home_games'] = len(home_games_df)
    teams.at[teams.team == team, 'away_games'] = len(away_games_df)
    teams.at[teams.team == team, 'neutral_games'] = len(neutral_games_df)

        # non dovrebbe essere necessario fare tutti e 3 i controlli ma data la presneza di errori nel dataset lo faccio comunque
    query1 = "home_team == @team and outcome == 'Home' or away_team == @team and outcome == 'Away' "
    teams.at[teams.team == team, 'wins'] = len( team_df.query(query1) )
    teams.at[teams.team == team, 'home_wins'] = len(home_games_df.query(query1))
    teams.at[teams.team == team, 'away_wins'] = len(away_games_df.query(query1))
    teams.at[teams.team == team, 'neutral_wins'] = len(neutral_games_df.query(query1))

    query2 = "home_team == @team and outcome == 'Away' or away_team == @team and outcome == 'Home' "
    teams.at[teams.team == team, 'defeats'] = len( team_df.query(query2) )
    teams.at[teams.team == team, 'home_defeats'] = len(home_games_df.query(query2))
    teams.at[teams.team == team, 'away_defeats'] = len(away_games_df.query(query2))
    teams.at[teams.team == team, 'neutral_defeats'] = len(neutral_games_df.query(query2))

    query3 = "outcome == 'Draw' "
    teams.at[teams.team == team, 'draws'] = len(team_df.query(query3))
    teams.at[teams.team == team, 'home_draws'] = len(home_games_df.query(query3))
    teams.at[teams.team == team, 'away_draws'] = len(away_games_df.query(query3))
    teams.at[teams.team == team, 'neutral_draws'] = len(neutral_games_df.query(query3))

    teams.at[teams.team == team, 'goals_scored'] = team_df.query("home_team == @team")['home_score'].sum() + team_df.query("away_team == @team")['away_score'].sum()
    teams.at[teams.team == team, 'home_goals_scored'] = home_games_df.query("home_team == @team")['home_score'].sum() + home_games_df.query("away_team == @team")['away_score'].sum()
    teams.at[teams.team == team, 'away_goals_scored'] = away_games_df.query("home_team == @team")['home_score'].sum() + away_games_df.query("away_team == @team")['away_score'].sum()
    teams.at[teams.team == team, 'neutral_goals_scored'] = neutral_games_df.query("home_team == @team")['home_score'].sum() + neutral_games_df.query("away_team == @team")['away_score'].sum()

    teams.at[teams.team == team, 'goals_conceded'] = team_df.query("home_team == @team")['away_score'].sum() + team_df.query("away_team == @team")['home_score'].sum()
    teams.at[teams.team == team, 'home_goals_conceded'] = home_games_df.query("home_team == @team")['away_score'].sum() + home_games_df.query("away_team == @team")['home_score'].sum()
    teams.at[teams.team == team, 'away_goals_conceded'] = away_games_df.query("home_team == @team")['away_score'].sum() + away_games_df.query("away_team == @team")['home_score'].sum()
    teams.at[teams.team == team, 'neutral_goals_conceded'] = neutral_games_df.query("home_team == @team")['away_score'].sum() + neutral_games_df.query("away_team == @team")['home_score'].sum()

    counter = counter + 1

teams.to_csv(path.join(PATH_DST, 'teams_statistics.csv'))

In [ ]:
plt.rcParams.update({'figure.figsize':(30,30), 'figure.dpi':100})
fig, axs = plt.subplots(4, 2)

df = pd.read_csv(path.join(PATH_DST, 'teams_statistics.csv')).drop(columns=['Unnamed: 0'])
colors = list(mcolors.XKCD_COLORS)
df['color'] = colors[:len(df)]

df_axs = df.sort_values(by=['games'], ascending=False)
axs[0, 0].set_title("Squadra che ha giocato di più")
axs[0, 0].barh(df_axs.team.values[:10], df_axs.games.values[:10], color=df_axs.color.values[:10], edgecolor='black')

df_axs = df.sort_values(by=['wins'], ascending=False)
axs[0, 1].set_title("Squadra che ha vinto di più")
axs[0, 1].barh(df_axs.team.values[:10], df_axs.wins.values[:10], color=df_axs.color.values[:10], edgecolor='black')

df_axs = df.sort_values(by=['defeats'], ascending=False)
axs[1, 0].set_title("Squadra che ha perso di più")
axs[1, 0].barh(df_axs.team.values[:10], df_axs.defeats.values[:10], color=df_axs.color.values[:10], edgecolor='black')

df_axs = df.sort_values(by=['draws'], ascending=False)
axs[1, 1].set_title("Squadra che ha pareggiato di più")
axs[1, 1].barh(df_axs.team.values[:10], df_axs.draws.values[:10], color=df_axs.color.values[:10], edgecolor='black')

df_axs = df.sort_values(by=['goals_scored'], ascending=False)
axs[2, 0].set_title("Squadra che ha segnato di più")
axs[2, 0].barh(df_axs.team.values[:10], df_axs.goals_scored.values[:10], color=df_axs.color.values[:10], edgecolor='black')

df_axs = df.sort_values(by=['goals_scored'], ascending=True)
axs[2, 1].set_title("Squadra che ha segnato di meno")
axs[2, 1].barh(df_axs.team.values[:10], df_axs.goals_scored.values[:10], color=df_axs.color.values[:10], edgecolor='black')

df_axs = df.sort_values(by=['goals_conceded'], ascending=False)
axs[3, 0].set_title("Squadra che ha concesso di più")
axs[3, 0].barh(df_axs.team.values[:10], df_axs.goals_conceded.values[:10], color=df_axs.color.values[:10], edgecolor='black')

df_axs = df.sort_values(by=['goals_conceded'], ascending=True)
axs[3, 1].set_title("Squadra che ha segnato di meno")
axs[3, 1].barh(df_axs.team.values[:10], df_axs.goals_conceded.values[:10], color=df_axs.color.values[:10], edgecolor='black')

plt.show()

correlazione di partite risultati di partite con goal fatti e subiti

In [ ]:
df = pd.read_csv(path.join(PATH_DST, 'teams_statistics.csv')).drop(columns=['Unnamed: 0','games', 'home_games', 'away_games',
    'neutral_games','home_wins', 'away_wins', 'neutral_wins','home_defeats', 'away_defeats', 'neutral_defeats','home_draws', 'away_draws', 'neutral_draws','home_goals_scored', 'away_goals_scored',
    'neutral_goals_scored','home_goals_conceded',
    'away_goals_conceded', 'neutral_goals_conceded'])

# sns.heatmap(df.corr(method='spearman'), annot=True)

plt.figure(figsize=(5, 5))
sns.heatmap(df.corr(method='spearman')[['goals_scored','goals_conceded']].sort_values(by=['goals_scored','goals_conceded'], ascending=False), vmin=-1, vmax=1, annot=True)

plt.show()

correlazione tra partite giocate in casa/ fuori casa e partite neutrali/away/home

In [ ]:
# df = pd.read_csv(path.join(PATH_DST, 'teams_statistics.csv')).drop(columns=['Unnamed: 0'])

# correlated = df.corr(method='spearman').unstack().sort_values(ascending=False).drop_duplicates()
# correlated[1:15]

# strong_corr = correlated[(correlated >= .7) & (correlated <= 1)]
# moderate_corr = correlated[(correlated >= .3) & (correlated <= .7)]
# weak_corr = correlated[(correlated >= .0) & (correlated <= .3)]

# print(strong_corr.__len__())
# print(moderate_corr.__len__())
# print(weak_corr.__len__())
# print(strong_corr)
# strong_corr.to_csv("output.csv")

df = pd.read_csv(path.join(PATH_DST, 'teams_statistics.csv')).drop(columns=['Unnamed: 0', 'confederation', 'games','wins','defeats','draws','goals_scored', 'home_goals_scored', 'away_goals_scored',
    'neutral_goals_scored', 'goals_conceded', 'home_goals_conceded',
    'away_goals_conceded', 'neutral_goals_conceded'])

# df.columns.values

sns.heatmap(df.corr(),annot=True)
plt.show()

## Parte 3 - Modello predittivo di goal fatti e subiti 

In [ ]:
df = pd.read_csv(path.join(PATH_DST, 'dataset_v3.csv')).drop(columns=['Unnamed: 0'])

### 3A - Visualizzazioni

Somma cumulativa dei risultati una specifica squadra

In [ ]:
plt.rcParams.update({'figure.figsize':(8,5), 'figure.dpi':100})
show_cdf(df, 'Italy')

In [ ]:
show_cdf_splitted(df, 'Italy')

Distribuzione dei goal segnati squadra di casa / squadra fuori casa

In [ ]:
plt.figure(figsize=(20,5))
sns.histplot(df[df['home_score'] > 0]['home_score'], kde=False, bins=30, label="Home score", color="y")
sns.histplot(df[df['away_score'] > 0]['away_score'], kde=False, bins=30, label="Away score", color="b")
# plt.xticks([i for i in range(1,15)])
plt.legend()
plt.title("Distribuzione di goals segnati")
plt.xlabel("Goals")
plt.ylabel("Frequenza")
plt.show()

Boxplot goal fatti / concessi rispettivamente nella squadra di casa e non

In [ ]:
# boxplot con i goal medi delle squadre

neutral_games = df.query("neutral == True")
not_neutral_games = df.query("neutral == False")

plt.figure(figsize=(20,5))
scores = {
    'home_score_NEUTRAL': neutral_games['home_score'], 'away_score_NEUTRAL': neutral_games['away_score'],
    'home_score_NOT_NEUTRAL': not_neutral_games['home_score'], 'away_score_NOT_NEUTRAL': not_neutral_games['away_score'],
    }
plt.boxplot(scores.values())

ax = plt.gca()
ax.set_xticklabels(scores.keys())

plt.show()

### 3B - Individuo la distribuzione e controllo la correlazione

In [ ]:
# confronto l'istogramma con la distribuzione di poisson per vedere se sono simili
plt.figure(figsize=(20,5))

plt.hist(df[['home_score', 'away_score']].values, range(6), alpha=0.8, label=['Home', 'Away'], density=True, color=["#6fa8dc", "#f6b26b"])

# sns.histplot(df[df['home_score'] > 0]['home_score'], kde=False, bins=30, label="Home score", color="y")
# sns.histplot(df[df['away_score'] > 0]['away_score'], kde=False, bins=30, label="Away score", color="b")
# plt.xticks([i for i in range(1,15)])

means = df[['home_score','away_score']].mean()
# construct Poisson  for each mean goals value
poisson_pred = np.column_stack([[poisson.pmf(k, means[j]) for k in range(8)] for j in range(2)])

plt.plot([i-0.5 for i in range(1,9)], poisson_pred[:,0],linestyle='-', marker='o',label="Home", color = '#6fa8dc')
plt.plot([i-0.5 for i in range(1,9)], poisson_pred[:,1],linestyle='-', marker='o',label="Away", color = '#f6b26b')

plt.legend()
plt.title("Distribuzione di goals segnati")
plt.xlabel("Goals")
plt.ylabel("Frequenza")
plt.show()

In [ ]:
# devo individuare la distribuzione notevole che più approssima il campione preso in esame.

# nota: voglio prevedere il numero di goal fatti dalla squadra home e quella away

# Supponiamo di sapere per certo che i pesi si distribuiscono secondo una distribuzione Gaussiana ed 
# effettuiamo il fit dei parametri. Confrontiamo dunque la Gaussiana trovata con l'istogramma di densità e la
# stima di densità del campione:

fig, axs = plt.subplots(2,1)
sns.despine(left=True)

sns.distplot(df['home_score'], ax=axs[0])
sns.distplot(df['away_score'], ax=axs[1])

plt.setp(axs, yticks=[])
plt.tight_layout()
plt.show()

print("Skewness home_score: %f" % df['home_score'].skew())
print("Skewness away_score: %f" % df['away_score'].skew())
print("Kurtosis home_score: %f" % df['home_score'].skew())
print("Kurtosis away_score: %f" % df['away_score'].skew())

# da notare l'asimmetria a dx e scriverlo.. e curtosi verso un certo punto.

In [ ]:
df['total_score'] = df['home_score'] + df['away_score']
sns.lmplot('total_score','away_score',data=df)
plt.show()

In [ ]:
sns.lmplot('total_score','away_score',data=df)
plt.show()

In [ ]:
sns.heatmap(df.drop(columns=['year']).corr(),annot=True)
plt.show()

### 3C - Rimuovo outliers

e vedo le differenze....

In [ ]:
df.describe()

In [ ]:
print(f"rows before removing: {df.shape[0]}")
df = remove_outliers(df, 'home_score')
df = remove_outliers(df, 'away_score')
print(f"rows after removing: {df.shape[0]}")

In [ ]:
df.describe()

In [ ]:
fig, axs = plt.subplots(2,1)
sns.despine(left=True)

sns.distplot(df['home_score'], ax=axs[0])
sns.distplot(df['away_score'], ax=axs[1])

plt.setp(axs, yticks=[])
plt.tight_layout()
plt.show()

print("Skewness home_score: %f" % df['home_score'].skew())
print("Skewness away_score: %f" % df['away_score'].skew())
print("Kurtosis home_score: %f" % df['home_score'].skew())
print("Kurtosis away_score: %f" % df['away_score'].skew())

In [ ]:
sns.lmplot('total_score','away_score',data=df)
plt.show()

In [ ]:
sns.lmplot('total_score','away_score',data=df)
plt.show()

In [ ]:
sns.heatmap(df.drop(columns=['year']).corr(),annot=True)
plt.show()

In [ ]:
df.reset_index(drop=True).to_csv(path.join(PATH_DST, "dataset_v4.csv"))

## d - Modello

In [ ]:
#### ----- completaaaa ------

# confronto con cdf
plt.figure(figsize=(10,6))

# weight home_score
cdf_hs = df['home_score'].value_counts(normalize=True).sort_index().cumsum()
cdf_hs.plot()

# poisson cdf predicted
poisson_pred = [poisson.pmf(k, means[0]) for k in cdf_hs.index.values]
#TODO

plt.plot()
plt.grid()
plt.legend(['CDF Campione','CDF Poisson'])
plt.show()

In [ ]:
# posso anche considerarmi il caso con outliers
# e con encoding
df = pd.read_csv(path.join(PATH_DST, 'dataset_v4.csv')).drop(columns=['Unnamed: 0'])

matches_model_data = pd.concat([df[['home_team','away_team','home_score']].rename(columns={'home_team':'team', 'away_team':'opponent','home_score':'goals'}),
            df[['away_team','home_team','away_score']].rename(columns={'away_team':'team', 'home_team':'opponent','away_score':'goals'})])

poisson_model = smf.glm(formula="goals ~ team + opponent", data=matches_model_data, 
                        family=sm.families.Poisson()).fit()

poisson_model.summary()

In [ ]:
print(get_match_result(poisson_model, 'Germany', 'Spain'))
print(get_match_result(poisson_model, 'Argentina', 'Germany'))
print(get_match_result(poisson_model, 'England', 'Morocco'))
print(get_match_result(poisson_model, 'Italy', 'Brazil'))